# Table Enforcer Demo

## Description


A python package to facilitate the iterative process of developing and using schema-like representations of DataFrames in pandas for recoding and validating instances of these data.

This is a very young attempt at solving a recurrent problem many people have.  So far I have looked at multiple solutions, but none really did it for me.

I need to load, recode, and validate tables all day everyday. Sometimes its simple; you can ``pandas.read_table()`` and all is good. But sometimes you have a 400 column long RedCap data dump that is complicated `af` and you need to develop your recoding logic through an iterative process.

This is an attempt to apply a sort of "test driven development" approach to data cleaning.


## Basic Workflow

1. For each column that you care about in your source table:
    1. Define a ``Column`` object that represents the ideal state of your data by passing a list of small, independent, reusable validator functions and some descriptive information.
    1. Use this object to validate the column data from your source table.
        - It will probably fail.
    1. Add small, composable, reusable recoding functions to the column object and iterate until your validations pass.
1. Define an ``Enforcer`` object by passing it a list of your column representation objects.
1. This enforcer can be used to recode or validate recoded tables of the same kind as your source table wherever your applications use that type of data.


## Near Future
Soon, I want to add more kinds of ``Column`` objects that implement one-to-many and many-to-one recoding logic as sometimes a column tries to do too much and should really be multiple columns as well as the reverse.


## Help
Please take a look and offer thoughts/advice.

https://github.com/xguse/table_enforcer


* Free software: MIT license
* Documentation: https://table-enforcer.readthedocs.io.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from table_enforcer import Enforcer, Column
import table_enforcer.errors as e

from table_enforcer import validate as v
from table_enforcer import recode as r

# Load or create your Table

In [3]:
table_path = "../tests/files/demo_table.csv"
df = pd.read_csv(table_path)
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


# Validator Functions

## Built-in Validators

Some validator functions come built-in (See the `table_enforcer/validate/funcs.py` module code for current list)

In [4]:
# ! cat $v.funcs.__file__

## Validator Call Signatures

In general, validators take a single `pandas.Series` object as input and return a `pandas.Series` of the same shape and indexes containing `True` or `False` relative to which items passed the validation logic. 

This changes a little if you are using a decorator like `@maxmin` (_See section on Decorating Validators_)



## Writing Custom Validators

In [5]:
def gte2(series):
    return series >= 2

def lte10(series):
    return series <= 10

def length_is_one(series):
    return series.str.len() == 1

def valid_sex(series):
    sex = set(['M', 'F'])
    return series.isin(sex)

## Decorating Validators
You can define decorators that extend the call signature of validators to accomadate more complex things like ranges.

Here we can define a single validator that accomplishes the same thing as the combination of the first two validators that we defined above:

In [6]:
@v.decorators.minmax(low=2, high=10)
def bt_2_and_10(series):
    """Test that the data items fall within range: 2 <= x <= 10."""
    return series

__Note:__ pay attention to the fact that here we return the __original__ series object. _We didn't do ANYTHING to it._ The testing gets done in the `@v.decorators.minmax` decorator function. This allows our validators to always expect a single argument at run-time and always return a single argument.  This is _very_ important.

### Writing Custom Validator Decorators

This is a bit beyond this demo's scope but here is the `table_enforcer/validate/decorators.py` code so that you can get an idea oh how to do it if you are brave.

In [7]:
# !cat $v.decorators.__file__

# Defining Columns

Now that we have a few validator functions, its time to put together a representation of our table and use that object to help us get us a final data table that is in a form we like. We will start with `col1`.

In [8]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


Lets say we don't care about `col2`; we want all the others though.

We create a column object for each column we want that contains the name of the column we want to represent. We set the options to describe our ideal format. We set the values for each option for each column including a a list of validator functions that should all pass for all items in each column if we got our way.

For now, we will ignore the `recoders` argument. What we end up with is the three column objects below.

In [9]:
col1 = Column(name='col1',
             dtype=np.int,
             unique=False,
             validators=[v.funcs.not_null, v.funcs.positive, bt_2_and_10],
             recoders=None)

col3 = Column(name='col3',
             dtype=np.int,
             unique=True,
             validators=[v.funcs.not_null, v.funcs.positive, bt_2_and_10],
             recoders=None)

col4 = Column(name='col4',
             dtype=str,
             unique=False,
             validators=[v.funcs.upper, length_is_one, valid_sex],
             recoders=None)

## Validating a Column object
For each column object we store the validator functions in a `dict`.

Here is the value for `col4`

In [10]:
col4.validators

{'length_is_one': <function __main__.length_is_one>,
 'upper': <function table_enforcer.validate.funcs.upper>,
 'valid_sex': <function __main__.valid_sex>}

We validate a column by passing our original dataframe's column to the column object. The tests get run independently and after the column data is passed through each test, we get a dataframe of results.  We can see right away which data items fail which tests.

This helps us plan our recoding efforts later. We can immediatly see what sorts of operations need to be run to convert those `False` tests to `True`.

In [11]:
col4.validate(df["col4"])

,length_is_one,upper,valid_sex,dtype
0,False,False,False,True
1,True,False,False,True
2,True,True,True,True


Wait a minute... Where did that validation test called `dtype` come from?

Turns out we get that one for free since we defined that this column needs to be of `dtype` `np.int`. We get a similar `unique` validation check inserted if we set that option to `True`. 

__NOTE:__
- To avoid confusion and frustration, the column's `series.name` is tested before validation and recoding to make it harder to accidently pass another table column into the wrong `Column.validate()` function. So it __must match the name argument we passed to our column object__.

In [12]:
col4.validate(df["col3"])

ValueError: The name of provided series 'col3' does not match this column's name 'col4'.

# Recoder Functions

Recoder functions have a very similar structure to validator functions. But their purpose is to coerce the raw data into a more useful form. For example `Col4` is obviously meant to represent 'male'/'female' logic. But we don't want to have to interpret all possible ways to represent that logic every time we want to use that column. So we write a recoder function to do convert all items to either "M" or "F" so we only need to understand two choices from now on.

In [13]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


## Built-in Recoders

Some recoder functions come built-in (See the `table_enforcer/recode/funcs.py` module code for current list)

In [14]:
# !cat $r.funcs.__file__

## Recoder Call Signatures

Like validators, recoders take a single `pandas.Series` object as input and return a `pandas.Series` of the same shape and indexes as the original series object. However, instead of returning a series of `True`/`False` values, it performs some operation on the data that gets the column data closer to being how you want it to look during analysis operations.

## Writing Custom Recoders

### Recoders are Meant to be Composable

You are able to build a single, monolithic recoder that transforms a column all the way into what you want in a single step. But it may be better to write recoders that perform only a single step on the way to getting a particular column into shape. 

This results in reusable functions that are "composable". Meaning that for the NEXT column, you may not even need to write a new function. All you may need to do is list a sequence of recoders that already exist. 

This is because, recoders are applied in a pipeline model.  The output from the first is supplied to the second etc.

Now, lets write our recoders to transform `col4`.

In [15]:
def standardize_sex(series):
    """Return a series where common representations of 'male'/'female' are standardized.
    
    Things like ['M', 'MALE', 'M', 'BOY', ...] are converted to `M`.
    Representations of female are treated similarly. 
    """
    mapper = {"M": "M",
              "MALE": "M",
              "BOY": "M",
              "F": "F",
              "FEMALE": "F",
              "GIRL": "F",
             }
    if series.str.islower().any():
        raise ValueError("standardize_sex expects input series to contain only UPPERCASE letters.")
    else:
        return series.apply(lambda x: mapper[x])

Since we plan to call the built-in recoder `upper` first, we only need to support uppercase text here. This is kind of a silly requirement in practice but it demonstrates how you can write composable recoders to fail if expectations are not met.

## Next Iteration: Validate Recoded Table

Lets revisit our original definition of `col4` and add the recoders we have now.

In [16]:
col4 = Column(name='col4',
             dtype=str,
             unique=False,
             validators=[v.funcs.upper, length_is_one, valid_sex],
             recoders=[r.funcs.upper, standardize_sex])

Now what do we have after we recode this column?

In [17]:
col4.recode(df['col4'])

0    M
1    M
2    F
Name: col4, dtype: object

That looks pretty good. But let's not rely on our eyes and instead rely on the validation tests we defined. 

Now we try `col4.recode` again but this time tell it to perform validation after recoding and see what happens.

If all is well, we will simply get the recoded column back. If validation fails, we will get a `ValidationError` raised.

In [18]:
col4.recode(df['col4'], validate=True)

0    M
1    M
2    F
Name: col4, dtype: object

We are good! We can now move on to our other columns and repeat the process until all the validation check pass on all of our columns. Then we can compose our `Enforcer` object and we will be nearing the end of our initial sanity check / recoding phase for this table. 

# Defining Table Enforcers

Just pass a list of the column objects we created to the `Enforcer`. The validation/recoding api for the `Enforcer` mirrors the one for `Columns` so you can reuse what we learned above.

In [19]:
demo = Enforcer(columns=[col1,col3,col4])

In [20]:
demo.columns

['col1', 'col3', 'col4']

In [21]:
demo.recode(df, validate=True)

,col1,col3,col4
0,7,10,M
1,2,6,M
2,6,2,F


__NOTE:__

- we will see what happens when the `demo.recode(df, validate=True)` encounters problems in recoding or validating when we pass some bad data to our `load_csv()` function below.

## Writing loading functions that use enforcers

Use these enforcers to recode or validate recoded tables of the same kind as your source table wherever your applications use that type of data like in your loading functions.

You can pass them into the function or reference them from the scope above as you would an imported function.

In [22]:
def load_csv(path, enforcer):
    df = pd.read_csv(path)
    return enforcer.recode(df, validate=True)

# Loading Good Data

In [23]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


In [24]:
df2 = load_csv(path=table_path, enforcer=demo)
df2

,col1,col3,col4
0,7,10,M
1,2,6,M
2,6,2,F


# Loading Bad Data 
In this case we load a CSV that is slightly different that the one we built our `enforcer` on: perhaps this is represents a new data dump or data from another source.

## The problems
1. `col1` includes a value less than 2 and we have a validator that requires that all values be between 2 and 10. 
    - Perhaps this is because this value represents some lab test that has a limit of detection of 2 so we can't confidently distinguish between 0-2.
    - So we want to set all values below the detection limit to the detection limit itself. However, we neglected to include a recoder for this since we didnt see these types of values in our first data-set.

2. `Feemale` is not found in the `standardize_sex()` recoder's `mapper` dictionary.
    - Here we encounter a value that actually has nothing to do with a validation check failing, but breaks our recoder itself.



In [25]:
table_path_bad = "../tests/files/demo_table_bad.csv"
df_bad = pd.read_csv(table_path_bad)
df_bad

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,1,6,2,F
3,6,5,5,Feemale


## ValidationErrors

A `ValidationError` is raised when we try to get a recoded table or column while setting the `recode` method's option `validate=True`.

Lets see what happens when we try to load this table with our original `demo` enforcer.

In [26]:
load_csv(path=table_path_bad, enforcer=demo)

ValidationError: Rows that failed to validate for column 'col1':
   bt_2_and_10  not_null  positive  dtype
2        False      True      True   True

A column object raises a `ValidationError` because one of it saw some data that did not pass validation while inside the `recode` method because we told it to validate. Also, it gives us some very helpful information:

1. The column name
2. The rows that did not pass validation
3. And which validations happened to fail on each row

Now we can write a recoder that deals with that value and move on.

In [27]:
# write another recoder
def handle_values_below_detection_limit(series):
    series[series < 2] = 2
    return series

# add the recoder to a Column representing `col1`
col1_new = Column(name='col1',
                  dtype=np.int,
                  unique=False,
                  validators=[v.funcs.not_null, v.funcs.positive, bt_2_and_10],
                  recoders=[handle_values_below_detection_limit])

# build enforcer with the corrected Column object
demo2 = Enforcer(columns=[col1_new,
                          col3,
                          col4])

Lets try this again with `demo2`.

In [28]:
df3 = load_csv(path=table_path_bad, enforcer=demo2)
df3

RecodingError: Recoder 'standardize_sex' raised the following error on column 'col4': KeyError('FEEMALE',).

__Problem 1__ seems to be fixed, but now we get This new `RecodingError`.

## RecodingErrors

A `RecodingError` is raised when a recoder function fails and raises an exception.

Like the `ValidationError` it tries to give you as much helpful information about where and how it failed.

In this case we need to update our recoder to deal with this new typo of the word "female"

In [29]:
# update the recoder
def standardize_sex2(series):
    """Return a series where common representations of 'male'/'female' are standardized.
    
    Things like ['M', 'MALE', 'M', 'BOY', ...] are converted to `M`.
    Representations of female are treated similarly. 
    """
    mapper = {"M": "M",
              "MALE": "M",
              "BOY": "M",
              "F": "F",
              "FEMALE": "F",
              "GIRL": "F",
              "FEEMALE": "F",
             }
    if series.str.islower().any():
        raise ValueError("standardize_sex expects input series to contain only UPPERCASE letters.")
    else:
        return series.apply(lambda x: mapper[x])

# add the recoder to a Column representing `col4`
col4_new = Column(name='col4',
                  dtype=str,
                  unique=False,
                  validators=[v.funcs.upper, length_is_one, valid_sex],
                  recoders=[r.funcs.upper, standardize_sex2])

# build enforcer with the corrected Column object
demo3 = Enforcer(columns=[col1_new,
                          col3,
                          col4_new])

OK one more time with `demo3` this time.

In [30]:
df4 = load_csv(path=table_path_bad, enforcer=demo3)
df4

,col1,col3,col4
0,7,10,M
1,2,6,M
2,2,2,F
3,6,5,F


Success! __Problem 2__ is no also fixed and we are ready to get back to work.


...until the next head ache.